In [2]:
import os
import Bio
import pandas as pd

In [120]:
wd = "."
input_id = "CMTR1_XChem_hits"
in_dir = os.path.join(wd, "input", input_id)
out_dir = os.path.join(wd, "output", input_id)
results_dir = os.path.join(out_dir, "results")

lig_data = pd.read_pickle(os.path.join(results_dir, "{}_lig_data.pkl".format(input_id)))

In [52]:
strucs = [f for f in os.listdir(in_dir) if f.endswith(".pdb") and "clean" not in f]
n_strucs = len(strucs)

In [60]:
struc2ligs = {}
for struc in strucs:
    struc_df = lig_data.query('struc_name == @struc')
    struc2ligs[struc] = []
    ligs = struc_df.label_comp_id.unique().tolist() 
    for the_lig in ligs: # RUNs ARPEGGIO ONCE FOR EACH LIGAND
        struc2ligs[struc].append(the_lig)

In [63]:
def dump_pickle(data, f_out):
    """
    dumps pickle
    """
    with open(f_out, "wb") as f:
        pickle.dump(data, f)

In [64]:
import pickle

In [68]:
struc2ligs

{'FabA-x0192-pandda-model.pdb': ['LIG'],
 'FabA-x0126-pandda-model.pdb': ['LIG'],
 'FabA-x0233-pandda-model.pdb': ['LIG'],
 'FabA-x0386-pandda-model.pdb': ['LIG'],
 'FabA-x0356-pandda-model.pdb': ['LIG'],
 'FabA-x0536-pandda-model.pdb': ['LIG'],
 '4B0C.pdb': ['C9H'],
 '7BHJ.pdb': ['TQH'],
 'FabA-x0495-pandda-model.pdb': [],
 'FabA-x0571-pandda-model.pdb': [],
 '4B0J.pdb': ['3MQ'],
 'FabA-x0388-pandda-model.pdb': ['LIG'],
 'FabA-x0539-pandda-model.pdb': [],
 'FabA-x0177-pandda-model.pdb': [],
 'FabA-x0431-pandda-model.pdb': ['LIG'],
 'FabA-x0416-pandda-model.pdb': ['LIG'],
 '7BIS.pdb': ['TZQ'],
 'FabA-x0570-pandda-model.pdb': ['LIG'],
 'FabA-x0142-pandda-model.pdb': ['LIG'],
 'FabA-x0503-pandda-model.pdb': ['LIG'],
 '7BK9.pdb': ['U0W'],
 '4B8U.pdb': ['IBK'],
 'FabA-x0119-pandda-model.pdb': ['LIG'],
 'FabA-x0162-pandda-model.pdb': ['LIG'],
 'FabA-x0507-pandda-model.pdb': ['LIG'],
 'FabA-x0266-pandda-model.pdb': ['LIG'],
 '4cl6.pdb': ['7SB'],
 'FabA-x0554-pandda-model.pdb': ['LIG'],
 'Fab

In [51]:
lig_data.query('struc_name == "7BHJ.pdb"')

,struc_name,label_comp_id,label_asym_id,auth_seq_id
11,7BHJ.pdb,TQH,B,201
12,7BHJ.pdb,TQH,C,201
13,7BHJ.pdb,TQH,D,201
14,7BHJ.pdb,TQH,E,201


In [5]:
def get_swissprot(): 
    """
    Retrieves sequences and their data from Swiss-Prot

    :param db: absolute path to a fasta file containing sequences, Swiss-Prot database by default
    :type db: str
    :returns: dictionary containing the sequence id, description and sequence for all proteins in Swiss-Prot
    :rtpe: dict
    """
    swissprot_dict = Bio.SeqIO.parse(swissprot, "fasta")
    proteins = {}
    for protein in swissprot_dict:
        acc = protein.id.split("|")[1]
        proteins[acc] = {}
        proteins[acc]["id"] = protein.id
        proteins[acc]["desc"] = protein.description
        proteins[acc]["seq"] = protein.seq
    return proteins

In [16]:
def retrieve_mapping_from_struc(struc, uniprot_id, struc_dir, sifts_dir, swissprot):
    input_struct = os.path.join(struc_dir, struc)
    pdb_structure = PDBXreader(inputfile = input_struct).atoms(format_type = "pdb") # ProIntVar reads the local file
    
    seq_record = str(swissprot[uniprot_id]["seq"])
    pps = pdb_structure[pdb_structure.group_PDB == "ATOM"][['label_comp_id', 'label_asym_id', 'label_seq_id_full']].drop_duplicates().groupby('label_asym_id')  # groupby chain
    pdb_chain_seqs = [(chain, SeqUtils.seq1(''.join(seq['label_comp_id'].values)), seq['label_seq_id_full'].values) for chain, seq in pps] # list of tuples like: [(chain_id, chain_seq, [chain resnums])]
    alignments = [pairwise2.align.globalxs(str(seq_record),chain_seq[1], -5, -1) for chain_seq in pdb_chain_seqs] # list of lists of tuples containing SwissProt seq - PDB chain seq pairwise alignment
    
    maps = []
    for pdb_chain_seq, alignment in zip(pdb_chain_seqs, alignments):
        PDB_UniProt_map = pd.DataFrame([(i, x) for i, x in enumerate(alignment[0][1], start=1)],  # create aligned PDB sequences to dataframe
                                       columns=['UniProt_ResNum', 'PDB_ResName'])
        PDB_UniProt_map = PDB_UniProt_map.assign(UniProt_ResName = list(alignment[0][0]))
        PDB_index = PDB_UniProt_map.query('PDB_ResName != "-"').index
        PDB_UniProt_map = PDB_UniProt_map.assign(PDB_ResNum = pd.Series(pdb_chain_seq[2], index=PDB_index)) # adds PDB_ResNum column
        PDB_UniProt_map = PDB_UniProt_map.assign(PDB_ChainID = pd.Series(pdb_chain_seq[0], index=PDB_index)) # adds PDB_ChainId column
        maps.append(PDB_UniProt_map)
    prointvar_mapping = pd.concat(maps)
    prointvar_mapping = prointvar_mapping[['UniProt_ResNum','UniProt_ResName','PDB_ResName','PDB_ResNum','PDB_ChainID']]
    prointvar_mapping = prointvar_mapping[~prointvar_mapping.PDB_ResNum.isnull()]
    prointvar_mapping.PDB_ResNum = prointvar_mapping.PDB_ResNum.astype(int)
    prointvar_mapping_csv = os.path.join(sifts_dir, "prointvar_mapping_" + struc + ".csv")
    prointvar_mapping.to_csv(prointvar_mapping_csv, index = False)
    return prointvar_mapping

In [9]:
import Bio.SeqIO

In [10]:
swissprot = "/cluster/gjb_lab/2394007/data/all_species/swissprot_rev_Nov21.fasta"

SP = get_swissprot()

In [65]:
dump_pickle(SP, "/cluster/gjb_lab/2394007/data/all_species/swissprot_rev_Nov21.pkl")

In [67]:
os.listdir("/cluster/gjb_lab/2394007/data/all_species/")

['seqs',
 'uniprot_anks_all_species.xml',
 'accs',
 'uniprot-keyword_ank+reviewed_yes.tab',
 'swissprot_rev_Nov21.fasta.bkp2',
 'uniprot_anks_all_species.tab',
 'swissprot_rev.fasta',
 'swissprot_rev_Nov21.pkl',
 'swissprot_rev_Nov21.fasta',
 'interpro_tsv_files_all_dbs',
 'uniprot_anks_all_species.txt',
 'swissprot_rev_Nov21.fasta.bkp1',
 'alns']

In [11]:
SP["O33877"]

{'id': 'sp|O33877|FABA_PSEAE',
 'desc': 'sp|O33877|FABA_PSEAE 3-hydroxydecanoyl-[acyl-carrier-protein] dehydratase OS=Pseudomonas aeruginosa (strain ATCC 15692 / DSM 22644 / CIP 104116 / JCM 14847 / LMG 12228 / 1C / PRS 101 / PAO1) OX=208964 GN=fabA PE=1 SV=1',
 'seq': Seq('MTKQHAFTREDLLRCSRGELFGPGNAQLPAPNMLMIDRIVHISDVGGKYGKGEL...DSF', SingleLetterAlphabet())}

In [21]:
from prointvar.pdbx import PDBXreader
from Bio import SeqUtils
from Bio import pairwise2

In [22]:
a = retrieve_mapping_from_struc(
    "FabA-x0569-pandda-model.pdb",
    "O33877",
    os.path.join(out_dir, "supp_pdbs"),
    os.path.join(out_dir, "sifts"),
    SP
)

2023-09-04 16:55:41,444 prointvar    INFO     Parsing PDB atoms from lines...
2023-09-04 16:55:42,026 prointvar    INFO     PDBx removed existing hydrogens...
2023-09-04 16:55:42,031 prointvar    INFO     PDBx reset atom numbers...


In [24]:
a.query('UniProt_ResName != PDB_ResName')

,UniProt_ResNum,UniProt_ResName,PDB_ResName,PDB_ResNum,PDB_ChainID


In [26]:
input_struct = os.path.join(os.path.join(out_dir, "supp_pdbs"), "FabA-x0569-pandda-model.pdb")
pdb_structure = PDBXreader(inputfile = input_struct).atoms(format_type = "pdb") # ProIntVar reads the local file

2023-09-05 08:30:21,223 prointvar    INFO     Parsing PDB atoms from lines...
2023-09-05 08:30:21,800 prointvar    INFO     PDBx removed existing hydrogens...
2023-09-05 08:30:21,807 prointvar    INFO     PDBx reset atom numbers...


In [29]:
uniprot_id = "O33877"

In [31]:
seq_record = str(SP[uniprot_id]["seq"])
pps = pdb_structure[pdb_structure.group_PDB == "ATOM"][['label_comp_id', 'label_asym_id', 'label_seq_id_full']].drop_duplicates().groupby('label_asym_id')  # groupby chain
pdb_chain_seqs = [(chain, SeqUtils.seq1(''.join(seq['label_comp_id'].values)), seq['label_seq_id_full'].values) for chain, seq in pps] # list of tuples like: [(chain_id, chain_seq, [chain resnums])]
alignments = [pairwise2.align.globalxs(str(seq_record),chain_seq[1], -5, -1) for chain_seq in pdb_chain_seqs] # list of lists of tuples containing SwissProt seq - PDB chain seq pairwise alignment

In [39]:
maps = []
for pdb_chain_seq, alignment in zip(pdb_chain_seqs, alignments):
    PDB_UniProt_map = pd.DataFrame([(i, x) for i, x in enumerate(alignment[0][1], start=1)],  # create aligned PDB sequences to dataframe
                                   columns=['UniProt_ResNum', 'PDB_ResName'])
    PDB_UniProt_map = PDB_UniProt_map.assign(UniProt_ResName = list(alignment[0][0]))
    PDB_index = PDB_UniProt_map.query('PDB_ResName != "-"').index
    PDB_UniProt_map = PDB_UniProt_map.assign(PDB_ResNum = pd.Series(pdb_chain_seq[2], index=PDB_index)) # adds PDB_ResNum column
    PDB_UniProt_map = PDB_UniProt_map.assign(PDB_ChainID = pd.Series(pdb_chain_seq[0], index=PDB_index)) # adds PDB_ChainId column
    maps.append(PDB_UniProt_map)

In [41]:
maps[0]

,UniProt_ResNum,PDB_ResName,UniProt_ResName,PDB_ResNum,PDB_ChainID
0,1,-,M,NaN,NaN
1,2,T,T,2,A
2,3,K,K,3,A
3,4,Q,Q,4,A
4,5,H,H,5,A
5,6,A,A,6,A
6,7,F,F,7,A
7,8,T,T,8,A
8,9,R,R,9,A
9,10,E,E,10,A


In [ ]:


prointvar_mapping = pd.concat(maps)
prointvar_mapping = prointvar_mapping[['UniProt_ResNum','UniProt_ResName','PDB_ResName','PDB_ResNum','PDB_ChainID']]
prointvar_mapping = prointvar_mapping[~prointvar_mapping.PDB_ResNum.isnull()]
#prointvar_mapping = prointvar_mapping.fillna("null")
prointvar_mapping.PDB_ResNum = prointvar_mapping.PDB_ResNum.astype(int)
prointvar_mapping_csv = os.path.join(sifts_dir, "prointvar_mapping_" + struc + ".csv")
prointvar_mapping.to_csv(prointvar_mapping_csv, index = False)
return prointvar_mapping

In [47]:
len(sorted(list(set([el.split(".")[0] for el in os.listdir(in_dir)]))))

46

In [82]:
a = pd.read_pickle("./output/CMTR1_XChem_hits/results/CMTR1_XChem_hits_BS_def_OC_complete_i_rel_0.66.pkl")

In [83]:
a

,pdb_id,lig_name,lig_resnum,lig_chain,binding_res,lab,binding_site,pdb_path,chimera_atom_spec
0,771463,ACT,602,A,"[156, 495, 498, 499, 502]",771463_ACT_602_A,1,771463.clean.pdb,:602.A&#/name==771463.clean.pdb
1,771463,ACT,603,A,"[172, 175, 459, 462, 463, 171]",771463_ACT_603_A,2,771463.clean.pdb,:603.A&#/name==771463.clean.pdb
2,771463,LIG,701,A,"[202, 205, 206, 209, 214, 511, 514, 515, 518, ...",771463_LIG_701_A,3,771463.clean.pdb,:701.A&#/name==771463.clean.pdb
3,771463,MLA,1,B,"[239, 275, 276, 277, 283, 362, 363, 364, 404, ...",771463_MLA_1_B,0,771463.clean.pdb,:1.B&#/name==771463.clean.pdb
4,CMTR1,MLA,1,B,"[239, 275, 276, 277, 283, 362, 363, 364, 403, ...",CMTR1_MLA_1_B,0,CMTR1.clean.pdb,:1.B&#/name==CMTR1.clean.pdb


In [84]:
dict(zip(a.lab,a.binding_site))

{'771463_ACT_602_A': 1,
 '771463_ACT_603_A': 2,
 '771463_LIG_701_A': 3,
 '771463_MLA_1_B': 0,
 'CMTR1_MLA_1_B': 0}

In [87]:
d = {k: sorted(v) for k, v in dict(zip(a.lab,a.binding_res)).items()}
d

{'771463_ACT_602_A': [156, 495, 498, 499, 502],
 '771463_ACT_603_A': [171, 172, 175, 459, 462, 463],
 '771463_LIG_701_A': [202, 205, 206, 209, 214, 511, 512, 514, 515, 518, 519],
 '771463_MLA_1_B': [239, 275, 276, 277, 283, 362, 363, 364, 403, 404, 443],
 'CMTR1_MLA_1_B': [239, 275, 276, 277, 283, 362, 363, 364, 403, 404, 441, 443]}

In [106]:
site_ress = {}
for site_id, site_rows in a.groupby("binding_site"):
    site_ress[site_id] = []
    for _, site_row in site_rows.iterrows():
        site_ress[site_id].extend(site_row.binding_res)
        
site_ress = {k: sorted(list(set(v))) for k, v in site_ress.items()}

In [107]:
site_ress

{0: [239, 275, 276, 277, 283, 362, 363, 364, 403, 404, 441, 443],
 1: [156, 495, 498, 499, 502],
 2: [171, 172, 175, 459, 462, 463],
 3: [202, 205, 206, 209, 214, 511, 512, 514, 515, 518, 519]}

In [112]:
site_ress = {0: [239, 275, 276, 277, 283, 362, 363, 364, 403, 404, 441, 443],
 1: [156, 239, 495, 498, 499, 502],
 2: [171, 172, 175, 459, 462, 463],
 3: [202, 205, 206, 209, 214, 511, 512, 514, 515, 518, 519]}

In [114]:
def get_residue_bs_membership(cluster_ress):
    """
    returns a dictionary indicating to which ligand binding
    site each ligand binding residue is found in. A residue
    might contribute to more than one adjacent binding site.
    """
    all_bs_ress = []
    for v in cluster_ress.values():
        all_bs_ress.extend(v)
    all_bs_ress = sorted(list(set(all_bs_ress)))
    
    bs_ress_membership_dict = {}
    for bs_res in all_bs_ress:
        bs_ress_membership_dict[bs_res] = []
        for k, v in cluster_ress.items():
            if bs_res in v:
                bs_ress_membership_dict[bs_res].append(k) # which binding site each residue belongs to
    
    return bs_ress_membership_dict

In [115]:
get_residue_bs_membership(site_ress)

{156: [1],
 171: [2],
 172: [2],
 175: [2],
 202: [3],
 205: [3],
 206: [3],
 209: [3],
 214: [3],
 239: [0, 1],
 275: [0],
 276: [0],
 277: [0],
 283: [0],
 362: [0],
 363: [0],
 364: [0],
 403: [0],
 404: [0],
 441: [0],
 443: [0],
 459: [2],
 462: [2],
 463: [2],
 495: [1],
 498: [1],
 499: [1],
 502: [1],
 511: [3],
 512: [3],
 514: [3],
 515: [3],
 518: [3],
 519: [3]}

In [77]:
b = pd.read_pickle("./output/CMTR1_XChem_hits/results/CMTR1_XChem_hits_ress_consvar.pkl")

In [79]:
b.head(3)

,UniProt_ResNum,alignment_column,shenkin,occ,gaps,occ_pct,gaps_pct,rel_norm_shenkin,abs_norm_shenkin,human_shenkin,human_occ,human_gaps,human_occ_pct,human_gaps_pct,variants,oddsratio,pvalue,se_OR
0,140,1,6.57,9,490,1.8,98.2,0.00,0.50,6.57,1,5,16.67,83.33,2,4.70,0.21,2.40
1,141,2,6.65,9,490,1.8,98.2,0.11,0.57,6.65,1,5,16.67,83.33,1,2.35,0.51,2.77
2,142,3,6.92,12,487,2.4,97.6,0.46,0.81,6.92,1,5,16.67,83.33,0,0.00,1.00,3.20


In [126]:
input_id = "CMTR1_XChem_hits"
df = pd.read_pickle(os.path.join(results_dir, "{}_dssp_mappings.pkl".format(input_id)))
df2 = pd.read_pickle(os.path.join(results_dir, "{}_dssp_mapped.pkl".format(input_id)))

In [127]:
df.equals(df2)

True

In [123]:
df.head(3)

,UniProt_ResNum,UniProt_ResName,PDB_ResName,PDB_ResNum,PDB_ChainID,CHAIN,AA,SS,ACC,TCO,KAPPA,ALPHA,PHI,PSI,CHAIN_FULL,RSA
0,140,N,N,140,A,A,N,NaN,217,0.000,360.0,360.0,360.0,124.1,A,138.217
1,141,V,V,141,A,A,V,NaN,63,-0.968,360.0,-124.0,-165.9,133.4,A,44.366
2,142,D,D,142,A,A,D,NaN,86,-0.728,35.9,158.9,-101.5,94.1,A,52.761


In [125]:
df.UniProt_ResNum.unique().tolist()

[140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 291,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 304,
 305,
 306

In [135]:
final_df = pd.read_pickle(os.path.join(results_dir, "{}_results_table.pkl".format(input_id)))

In [136]:
final_df.columns.tolist()

['UniProt_ResNum',
 'alignment_column',
 'shenkin',
 'occ',
 'gaps',
 'occ_pct',
 'gaps_pct',
 'rel_norm_shenkin',
 'abs_norm_shenkin',
 'human_shenkin',
 'human_occ',
 'human_gaps',
 'human_occ_pct',
 'human_gaps_pct',
 'variants',
 'oddsratio',
 'pvalue',
 'se_OR',
 'AA',
 'RSA',
 'SS',
 'binding_sites']

In [137]:
final_df.query('binding_sites == binding_sites')

,UniProt_ResNum,alignment_column,shenkin,occ,gaps,occ_pct,gaps_pct,rel_norm_shenkin,abs_norm_shenkin,human_shenkin,...,human_occ_pct,human_gaps_pct,variants,oddsratio,pvalue,se_OR,AA,RSA,SS,binding_sites
16,156,19,6.92,12,487,2.40,97.60,0.46,0.81,6.92,...,16.67,83.33,0,0.00,1.00,3.20,S,60.26,C,[1]
31,171,40,7.02,13,486,2.61,97.39,0.59,0.89,7.02,...,16.67,83.33,1,2.35,0.51,2.77,M,1.95,T,[2]
32,172,41,7.00,13,486,2.61,97.39,0.57,0.88,7.00,...,16.67,83.33,0,0.00,1.00,3.20,S,75.13,T,[2]
35,175,45,6.87,13,486,2.61,97.39,0.39,0.76,6.87,...,16.67,83.33,2,4.70,0.21,2.40,M,32.62,C,[2]
62,202,72,8.20,29,470,5.81,94.19,2.14,1.93,8.20,...,16.67,83.33,1,2.35,0.51,2.77,C,27.16,H,[3]
65,205,75,8.27,30,469,6.01,93.99,2.24,1.99,8.27,...,16.67,83.33,1,2.35,0.51,2.77,V,69.01,H,[3]
66,206,76,8.21,30,469,6.01,93.99,2.16,1.94,8.21,...,16.67,83.33,1,2.35,0.51,2.77,F,13.71,H,[3]
69,209,86,9.24,43,456,8.62,91.38,3.51,2.84,9.24,...,16.67,83.33,0,0.00,1.00,3.20,L,34.35,S,[3]
74,214,91,11.12,60,439,12.02,87.98,5.98,4.49,11.12,...,33.33,66.67,0,0.00,1.00,3.04,M,9.93,H,[3]
99,239,126,10.51,381,118,76.35,23.65,5.18,3.96,10.51,...,66.67,33.33,1,0.59,1.00,2.19,K,17.72,H,[0]
